##### 作業用ディレクトリの準備
- ローカルのWindows環境で実行する場合は，Git for Windowsを事前にインストールしておく必要があります．

In [ ]:
import os


# 独自ライブラリ等のダウンロード
if not os.path.isdir('AI_advanced'):
    !git clone https://github.com/knakamura1982/AI_advanced.git
%cd AI_advanced

# モデルファイルの保存先ディレクトリの作成
if not os.path.isdir('CGAN_models'):
    !mkdir CGAN_models

# 一時ファイルの保存先ディレクトリの作成
if os.path.exists('temp'):
    if os.name == 'nt':
        !Powershell.exe -Command "rm -r -fo temp"
    else:
        !rm -fr temp
!mkdir temp

##### 顔画像データセットCelebAのダウンロード・解凍
- **前回の試行の続きを行いたい場合（再開モードの場合）でも実行が必要です．**
- これは本来のCelebAではなく，その中から10%弱の画像をランダムに抜き出した簡易版です．
- 十数分かかる可能性があります．

In [ ]:
import os
import torch
from torchvision import transforms
from mylib.data_io import CSVBasedDataset


if not os.path.isfile('./Datasets/tinyCelebA_train_images.pt'):
    if os.name == 'nt':
        # ローカルのWindows環境の場合
        !Powershell.exe -Command "wget https://tus.box.com/shared/static/z7a4pb9qtco6fwspige2tpt2ryhqv9l1.gz -O tinyCelebA.tar.gz"
        !Powershell.exe -Command "tar -zxf tinyCelebA.tar.gz"
        !Powershell.exe -Command "rm -fo tinyCelebA.tar.gz"
    else:
        # それ以外（Colab環境含む）の場合
        !wget "https://tus.box.com/shared/static/z7a4pb9qtco6fwspige2tpt2ryhqv9l1.gz" -O tinyCelebA.tar.gz
        !tar -zxf tinyCelebA.tar.gz
        !rm -f tinyCelebA.tar.gz
    dataset = CSVBasedDataset(
        dirname='./tinyCelebA',
        filename='./tinyCelebA/image_list.csv',
        items=[
            'File Path',
            [
                '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry',
                'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
                'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair',
                'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young'
            ]
        ],
        dtypes=['image', 'float'],
        img_transform=transforms.CenterCrop((128, 128))
    )
    data = [dataset[i] for i in range(len(dataset))]
    image_tensor = torch.cat([torch.unsqueeze(u, dim=0) for u, v in data], dim=0)
    label_tensor = torch.cat([torch.unsqueeze(v, dim=0) for u, v in data], dim=0)
    torch.save(image_tensor, './Datasets/tinyCelebA_train_images.pt')
    torch.save(label_tensor, './Datasets/tinyCelebA_train_labels.pt')
    del dataset, data, image_tensor, label_tensor
    if os.name == 'nt':
        !Powershell.exe -Command "rm -r -fo tinyCelebA"
    else:
        !rm -fr tinyCelebA

##### 必要なら，自分のgoogle driveに退避させておいた過去の学習経過情報をロード
- Google Colab以外の環境で使用することは想定していません．

In [ ]:
import os
from google.colab import drive


# 自分のgoogle driveにおける退避先フォルダの名称
DST_DIR_NAME = 'AI_advanced_temp'

# 退避先フォルダの中身をtempフォルダにまるごとコピー
MOUNT_POINT = '/content/drive'
DST_DIR_PATH = os.path.join(MOUNT_POINT, 'MyDrive', DST_DIR_NAME)
if not os.path.isdir(MOUNT_POINT):
    drive.mount(MOUNT_POINT)
!cp -r $DST_DIR_PATH/* temp

##### データセットの場所やバッチサイズなどの定数値の設定

In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
import torch


# 前回の試行の続きを行いたい場合は True にする -> 再開モードになる．
# なお，Colab環境で再開モードを利用する場合は，前回終了時に temp ディレクトリの中身を自分の Google Drive に退避しておき，
# それを改めて /content/AI_advanced/temp 以下にあらかじめ移しておく必要がある．
RESTART_MODE = True


# 使用するデバイス
# GPU を使用しない環境（CPU環境）で実行する場合は DEVICE = 'cpu' とする．
# GPU が複数存在する環境では，'cuda:0', 'cuda:1', 'cuda:2' などのような形で使用するGPUのIDを指定する．
# Google Colab, Paperspace Gradient などで GPU を利用する場合は DEVICE = 'cuda:0' とすれば良いはず．
DEVICE = 'cuda:0'

# 高速化・省メモリ化のために半精度小数を用いた混合精度学習を行うか否か（Trueの場合は行う）
USE_AMP = True
FLOAT_DTYPE = torch.float16 # 混合精度学習を行う場合の半精度小数の型．環境によっては torch.bfloat16 にした方が良好な性能になる（ただしColabのT4 GPU環境ではムリ）．

# 混合精度学習の設定
if DEVICE == 'cpu':
    USE_AMP = False # CPU使用時は強制的に混合精度学習をOFFにする
LOSS_SCALER = torch.amp.grad_scaler.GradScaler(enabled=USE_AMP, device='cuda', init_scale=2**12)
ADAM_EPS = 1e-4 if USE_AMP and (FLOAT_DTYPE == torch.float16) else 1e-8

# 全ての訓練データを一回ずつ使用することを「1エポック」として，何エポック分学習するか
N_EPOCHS = 100

# 学習時のバッチサイズ
BATCH_SIZE = 64

# データセットの存在するフォルダ・ファイル名
DATA_DIR = './Datasets/'
TRAIN_IMAGES_FILE = 'tinyCelebA_train_images.pt'
TRAIN_LABELS_FILE = 'tinyCelebA_train_labels.pt'

# tinyCelebAにおける属性ラベルの名称と番号の対応表
ATTRIBUTE_TABLE = {
    '5_o_Clock_Shadow'  :  0, 'Arched_Eyebrows'     :  1, 'Attractive'       :  2, 'Bags_Under_Eyes' :  3,
    'Bald'              :  4, 'Bangs'               :  5, 'Big_Lips'         :  6, 'Big_Nose'        :  7,
    'Black_Hair'        :  8, 'Blond_Hair'          :  9, 'Blurry'           : 10, 'Brown_Hair'      : 11,
    'Bushy_Eyebrows'    : 12, 'Chubby'              : 13, 'Double_Chin'      : 14, 'Eyeglasses'      : 15,
    'Goatee'            : 16, 'Gray_Hair'           : 17, 'Heavy_Makeup'     : 18, 'High_Cheekbones' : 19,
    'Male'              : 20, 'Mouth_Slightly_Open' : 21, 'Mustache'         : 22, 'Narrow_Eyes'     : 23,
    'No_Beard'          : 24, 'Oval_Face'           : 25, 'Pale_Skin'        : 26, 'Pointy_Nose'     : 27,
    'Receding_Hairline' : 28, 'Rosy_Cheeks'         : 29, 'Sideburns'        : 30, 'Smiling'         : 31,
    'Straight_Hair'     : 32, 'Wavy_Hair'           : 33, 'Wearing_Earrings' : 34, 'Wearing_Hat'     : 35,
    'Wearing_Lipstick'  : 36, 'Wearing_Necklace'    : 37, 'Wearing_Necktie'  : 38, 'Young'           : 39
}

# 取り扱う属性ラベル（上表の中から名称で指定）
TARGET_ATTRIBUTES = ['Blond_Hair', 'Brown_Hair', 'Black_Hair', 'Gray_Hair', 'Eyeglasses', 'Male', 'Young']

# 取り扱う属性ラベルの番号
TARGET_ATTRIBUTES_ID = [ATTRIBUTE_TABLE[a] for a in TARGET_ATTRIBUTES]

# 画像サイズ
H = 128 # 縦幅
W = 128 # 横幅
C = 3 # チャンネル数（カラー画像なら3，グレースケール画像なら1）

# 特徴ベクトルの次元数
N = 128

# 学習結果の保存先フォルダ
MODEL_DIR = './CGAN_models/'

# 学習結果のニューラルネットワークの保存先
MODEL_FILE_G = os.path.join(MODEL_DIR, './face_generator_model.pth') # ジェネレータ
MODEL_FILE_D = os.path.join(MODEL_DIR, './face_discriminator_model.pth') # ディスクリミネータ

# 中断／再開の際に用いる一時ファイル
CHECKPOINT_EPOCH = os.path.join('./temp/', 'checkpoint_epoch.pkl')
CHECKPOINT_GEN_MODEL = os.path.join('./temp/', 'checkpoint_gen_model.pth')
CHECKPOINT_DIS_MODEL = os.path.join('./temp/', 'checkpoint_dis_model.pth')
CHECKPOINT_GEN_OPT = os.path.join('./temp/', 'checkpoint_gen_opt.pth')
CHECKPOINT_DIS_OPT = os.path.join('./temp/', 'checkpoint_dis_opt.pth')

##### ニューラルネットワークモデルの定義

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from mylib.basic_layers import Reshape, MinibatchDiscrimination, DiscriminatorAugmentation


# Pre-act Residual Block
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, sn=False):
        super(ResBlock, self).__init__()
        shortcut_conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        main_conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        main_conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        if sn:
            # spectral normalization を用いる場合（主にディスクリミネータ用）
            self.shortcut = nn.utils.spectral_norm(shortcut_conv)
            self.block1 = nn.Sequential(nn.ReLU(), nn.utils.spectral_norm(main_conv1))
            self.block2 = nn.Sequential(nn.ReLU(), nn.utils.spectral_norm(main_conv2))
        else:
            # バッチ正規化を用いる場合（主にジェネレータ用）
            self.shortcut = shortcut_conv
            self.block1 = nn.Sequential(nn.BatchNorm2d(num_features=in_channels), nn.ReLU(), main_conv1)
            self.block2 = nn.Sequential(nn.BatchNorm2d(num_features=out_channels), nn.ReLU(), main_conv2)
    def forward(self, x):
        s = self.shortcut(x)
        h = self.block1(x)
        h = self.block2(h)
        return h + s


# GANジェネレータ用のアップサンプリング層
class myUpsamplingBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(myUpsamplingBlock, self).__init__()
        self.up = nn.UpsamplingNearest2d(scale_factor=2)
        self.rb = ResBlock(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1, sn=False)
    def forward(self, x):
        h = self.up(x)
        return self.rb(h)


# GANディスクリミネータ用のダウンサンプリング層
class myDownsamplingBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(myDownsamplingBlock, self).__init__()
        self.rb = ResBlock(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1, sn=True)
        self.down = nn.AvgPool2d(kernel_size=2)
    def forward(self, x):
        h = self.rb(x)
        return self.down(h)


# 顔画像生成ニューラルネットワーク
# CGAN生成器（ジェネレータ）のサンプル
class Generator(nn.Module):

    # C: 出力顔画像のチャンネル数（1または3と仮定）
    # H: 出力顔画像の縦幅（32の倍数と仮定）
    # W: 出力顔画像の横幅（32の倍数と仮定）
    # N: 入力の特徴ベクトル（乱数ベクトル）の次元数
    # K: 属性ラベルの種類数
    def __init__(self, C, H, W, N, K):
        super(Generator, self).__init__()
        self.W = W
        self.H = H

        # 属性ラベル情報を処理する全結合層
        self.embed = nn.Linear(in_features=K, out_features=N) # 属性ラベル情報を特徴ベクトルと同じ N 次元に拡張

        # 属性ラベル情報と特徴ベクトルを連結した後のベクトルをチャンネル数 512, 縦幅 H/32, 横幅 W/32 の特徴マップに変換する層
        self.conv0 = nn.Sequential(
            Reshape(size=(2*N, 1, 1)),
            nn.ConvTranspose2d(in_channels=2*N, out_channels=512, kernel_size=(H//32, W//32), stride=1, padding=0),
        )

        # アップサンプリング層1～5
        # これらを通すことにより特徴マップの縦幅・横幅がそれぞれ 2 倍になる
        # 5つ通すことになるので，最終的には都合 32 倍になる -> ゆえに縦幅 H/32, 横幅 W/32 の特徴マップからスタートする
        self.up1 = myUpsamplingBlock(in_channels=512, out_channels=256)
        self.up2 = myUpsamplingBlock(in_channels=256, out_channels=128)
        self.up3 = myUpsamplingBlock(in_channels=128, out_channels=64)
        self.up4 = myUpsamplingBlock(in_channels=64, out_channels=32)
        self.up5 = myUpsamplingBlock(in_channels=32, out_channels=32)

        # 出力画像生成用の畳込み層
        self.conv5 = nn.Sequential(
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=C, kernel_size=1, stride=1, padding=0),
        )

    def forward(self, z, y):
        y = self.embed(y) # 属性ラベル情報を N 次元に
        h = torch.cat((z, y), dim=1) # 特徴ベクトルと属性ラベル情報を連結 -> トータル256次元に
        h = self.conv0(h) # 256次元の特徴ベクトルをチャンネル数 512, 縦幅 H/32, 横幅 W/32 の特徴マップに変換
        h = self.up1(h)
        h = self.up2(h)
        h = self.up3(h)
        h = self.up4(h)
        h = self.up5(h)
        y = torch.tanh(self.conv5(h))
        return y


# 顔画像が Real か Fake を判定するニューラルネットワーク
# CGAN識別器（ディスクリミネータ）のサンプル
class Discriminator(nn.Module):

    # C: 入力顔画像のチャンネル数（1または3と仮定）
    # H: 入力顔画像の縦幅（32の倍数と仮定）
    # W: 入力顔画像の横幅（32の倍数と仮定）
    # K: 属性ラベルの種類数
    def __init__(self, C, H, W, K):
        super(Discriminator, self).__init__()

        # 訓練データ量の不足を補うためのデータ拡張（Data Augmentation）処理
        self.preprocess = DiscriminatorAugmentation(H, W, p_hflip=0.5, p_vflip=0.4, p_rot=0.4) # 確率0.5で左右反転，確率0.4で上下反転，確率0.4で回転

        # ダウンサンプリング層1～5
        # カーネルサイズ4，ストライド幅2，パディング1の設定なので，これらを通すことにより特徴マップの縦幅・横幅がそれぞれ 1/2 になる
        self.down1 = myDownsamplingBlock(in_channels=C+K, out_channels=32)
        self.down2 = myDownsamplingBlock(in_channels=32, out_channels=64)
        self.down3 = myDownsamplingBlock(in_channels=64, out_channels=128)
        self.down4 = myDownsamplingBlock(in_channels=128, out_channels=256)
        self.down5 = myDownsamplingBlock(in_channels=256, out_channels=256)

        # 平坦化
        self.flat = nn.Flatten()

        # 全結合層1（spectral normalization を使用）
        # ダウンサンプリング層1～5を通すことにより特徴マップの縦幅・横幅は都合 1/32 になっているので，
        # 入力側のパーセプトロン数は 256*(H/32)*(W/32) = H*W/4
        self.fc1 = nn.utils.spectral_norm(nn.Linear(in_features=H*W//4, out_features=256))

        # 全結合層2
        self.fc2 = nn.Linear(in_features=384, out_features=1)

        # Minibatch Discrimination: モード崩壊を回避するための技法の一つ
        self.md = MinibatchDiscrimination(in_features=256, out_features=128)

    def forward(self, x, y):
        # 本来であれば，ディスクリミネータの出力が 0～1 の範囲となるよう，最終層の活性化関数として sigmoid を適用すべきであるが，
        # このサンプルコードでは損失関数側で sigmoid 適用することになるので, ここでは最終層で活性化関数を適用しない
        y = y.reshape(*y.size(), 1, 1).repeat(1, 1, x.size()[2], x.size()[3]) # 属性ラベル情報を画像と同じ形に拡張
        x = self.preprocess(x)
        h = torch.cat((x, y), dim=1) # 画像情報とラベル情報を結合
        h = self.down1(h)
        h = self.down2(h)
        h = self.down3(h)
        h = self.down4(h)
        h = self.down5(h)
        h = self.flat(h)
        h = F.relu(self.fc1(h))
        h = self.md(h) # Minibatch Discrimination
        z = self.fc2(h) # 上記の通り，最終層では活性化関数なし
        return z

##### 訓練データセットの読み込み

In [ ]:
import torch
from torch.utils.data import DataLoader
from mylib.data_io import TensorDataset


# テンソルファイルを読み込み, 訓練データセットを用意
# 今回は，全てのデータを学習用に回す
image_tensor = torch.load(os.path.join(DATA_DIR, TRAIN_IMAGES_FILE), weights_only=True)
label_tensor = torch.load(os.path.join(DATA_DIR, TRAIN_LABELS_FILE), weights_only=True)
label_tensor = torch.cat([label_tensor[:, TARGET_ATTRIBUTES_ID[i]:TARGET_ATTRIBUTES_ID[i]+1] for i in range(len(TARGET_ATTRIBUTES_ID))], dim=1)
train_dataset = TensorDataset(tensors=[image_tensor, label_tensor])
train_size = len(train_dataset)

# 訓練データをミニバッチに分けて使用するための「データローダ」を用意
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

##### 学習処理の実行
- CGANの学習は一般に安定せず，最終的なモデルよりも学習途中のモデルの方が優れていることがよくあります
- このため，エポックごとにモデル保存処理を実行し，学習終了後，最良（と思われる）モデルをロードして利用することも多いです
- ただし，これを Google Colab などのクラウド環境で実行するとストレージ使用量の上限を超えてしまう可能性があるので，注意してください

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from mylib.loss_functions import GANLoss
from mylib.visualizers import LossVisualizer
from mylib.data_io import show_images, to_sigmoid_image, to_tanh_image, autosaved_model_name
from mylib.utility import save_checkpoint, load_checkpoint


# 何エポックに1回の割合で学習経過を表示するか（モデル保存処理もこれと同じ頻度で実行）
INTERVAL_FOR_SHOWING_PROGRESS = 10

# spectral normalization の使用によりディスクリミネータが弱体化するので，ジェネレータの更新回数を減らすことが望ましいらしいが，実際にはなんとも言い難い
# ここでは，ジェネレータを5回に1回の割合で更新することにする
N_DIS = 5 # この値を 1 にすれば，ジェネレータも毎回更新されるようになる


# エポック番号
INIT_EPOCH = 0 # 初期値
LAST_EPOCH = INIT_EPOCH + N_EPOCHS # 最終値

# ニューラルネットワークの作成
gen_model = Generator(C=C, H=H, W=W, N=N, K=len(TARGET_ATTRIBUTES)).to(DEVICE)
dis_model = Discriminator(C=C, H=H, W=W, K=len(TARGET_ATTRIBUTES)).to(DEVICE)

# 最適化アルゴリズムの指定（ここでは SGD でなく Adam を使用）
gen_optimizer = optim.Adam(gen_model.parameters(), lr=0.0002, betas=(0.5, 0.999), eps=ADAM_EPS)
dis_optimizer = optim.Adam(dis_model.parameters(), lr=0.0002, betas=(0.5, 0.999), eps=ADAM_EPS)

# 再開モードの場合は，前回チェックポイントから情報をロードして学習再開
if RESTART_MODE:
    INIT_EPOCH, LAST_EPOCH, gen_model, gen_optimizer = load_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_GEN_MODEL, CHECKPOINT_GEN_OPT, N_EPOCHS, gen_model, gen_optimizer)
    _, _, dis_model, dis_optimizer = load_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_DIS_MODEL, CHECKPOINT_DIS_OPT, N_EPOCHS, dis_model, dis_optimizer)
    print('')

# 損失関数
loss_func = GANLoss(label_smoothing=True)

# 検証の際に使用する乱数ベクトルおよび属性ラベル情報を用意
Z_valid = torch.randn(BATCH_SIZE, N).to(DEVICE) # 検証用乱数ベクトル
L_valid = torch.zeros(BATCH_SIZE, len(TARGET_ATTRIBUTES)).to(DEVICE) # 検証用属性ラベル情報

# 損失関数値を記録する準備
loss_viz = LossVisualizer(['G loss', 'D loss'], init_epoch=INIT_EPOCH)

# 勾配降下法による繰り返し学習
for epoch in range(INIT_EPOCH, LAST_EPOCH):

    print('Epoch {0}:'.format(epoch + 1))

    # 学習
    gen_model.train()
    dis_model.train()
    sum_gen_loss = 0
    sum_dis_loss = 0
    n_iter = 1 # 1エポック内でのループ回数を記録する変数（ジェネレータの更新回数を制御するために使用）
    for X, L in tqdm(train_dataloader):
        for param in gen_model.parameters():
            param.grad = None
        for param in dis_model.parameters():
            param.grad = None
        L = L.to(DEVICE) # 属性ラベル情報
        Z = torch.randn(len(X), N).to(DEVICE) # 乱数ベクトルを用意
        real = to_tanh_image(X).to(DEVICE) # Real画像を用意（to_tanh_image 関数を用い，画素値の範囲が -1〜1 となるように調整しておく）
        with torch.amp.autocast_mode.autocast(enabled=USE_AMP, device_type='cuda', dtype=FLOAT_DTYPE):
            fake = gen_model(Z, L) # Fake画像を生成（2行上で用意した Z から生成）
            fake_cpy = fake.detach() # Fake画像のコピーを用意しておく
            ### ジェネレータの学習 ###
            if n_iter % N_DIS == 0:
                Y_fake = dis_model(fake, L) # Fake画像を識別
                gen_loss = loss_func.G_loss(Y_fake)
                LOSS_SCALER.scale(gen_loss).backward()
                LOSS_SCALER.step(gen_optimizer)
                LOSS_SCALER.update()
                sum_gen_loss += float(gen_loss) * len(X)
            ### ディスクリミネータの学習 ###
            for param in dis_model.parameters():
                param.grad = None # ジェネレータの学習時の計算した勾配を一旦リセット
            Y_real = dis_model(real, L) # Real画像を識別
            Y_fake = dis_model(fake_cpy, L) # Fake画像を識別（コピー変数の方を使用）
            dis_loss = loss_func.D_loss(Y_fake, as_real=False) + loss_func.D_loss(Y_real, as_real=True)
            LOSS_SCALER.scale(dis_loss).backward()
            LOSS_SCALER.step(dis_optimizer)
            LOSS_SCALER.update()
            sum_dis_loss += float(dis_loss) * len(X)
        n_iter += 1
    avg_gen_loss = sum_gen_loss * N_DIS / train_size
    avg_dis_loss = sum_dis_loss / train_size
    loss_viz.add_value('G loss', avg_gen_loss) # 訓練データに対する損失関数の値を記録
    loss_viz.add_value('D loss', avg_dis_loss) # 同上
    print('generator train loss = {0:.6f}'.format(avg_gen_loss))
    print('discriminator train loss = {0:.6f}'.format(avg_dis_loss))
    print('')

    # 検証（学習経過の表示，モデル自動保存）
    if epoch == 0 or (epoch + 1) % INTERVAL_FOR_SHOWING_PROGRESS == 0:
        gen_model.eval()
        dis_model.eval()
        if epoch == 0:
            real = to_sigmoid_image(real) # to_sigmoid_image 関数を用い，画素値が 0〜1 の範囲となるように調整する
            show_images(real.to('cpu').detach(), num=32, num_per_row=8, title='real images', save_fig=False, save_dir=MODEL_DIR)
        with torch.inference_mode():
            fake = gen_model(Z_valid, L_valid) # 事前に用意しておいた検証用乱数と属性ラベル情報からFake画像を生成
            #fake = gen_model(torch.randn(BATCH_SIZE, N).to(DEVICE), torch.zeros(BATCH_SIZE, len(TARGET_ATTRIBUTES)).to(DEVICE)) # エポックごとに異なる乱数を使用する場合はこのようにする
        fake = to_sigmoid_image(fake) # to_sigmoid_image 関数を用い，画素値が 0〜1 の範囲となるように調整する
        show_images(fake.to('cpu').detach(), num=32, num_per_row=8, title='epoch {0}'.format(epoch + 1), save_fig=False, save_dir=MODEL_DIR)
        torch.save(gen_model.state_dict(), autosaved_model_name(MODEL_FILE_G, epoch + 1)) # 学習途中のモデルを保存したい場合はこのようにする

    # 現在の学習状態を一時ファイル（チェックポイント）に保存
    save_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_GEN_MODEL, CHECKPOINT_GEN_OPT, epoch+1, gen_model, gen_optimizer)
    save_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_DIS_MODEL, CHECKPOINT_DIS_OPT, epoch+1, dis_model, dis_optimizer)

# 学習結果のニューラルネットワークモデルをファイルに保存
gen_model = gen_model.to('cpu')
dis_model = dis_model.to('cpu')
torch.save(gen_model.state_dict(), MODEL_FILE_G)
#torch.save(dis_model.state_dict(), MODEL_FILE_D) # ディスクリミネータも保存したい場合はこのようにする

# 損失関数の記録をファイルに保存
loss_viz.save(v_file=os.path.join(MODEL_DIR, 'loss_graph.png'), h_file=os.path.join(MODEL_DIR, 'loss_history.csv'))

##### 必要なら，現在の学習経過情報を自分のgoogle driveに退避
- Google Colab以外の環境で使用することは想定していません．

In [ ]:
import os
from google.colab import drive


# 退避先フォルダの名称
# 自分のgoogle driveにおけるルートフォルダの直下にこの名前のフォルダが作成される
# 既に存在する場合は一度削除した上で再作成される（削除したフォルダはgoogle driveのゴミ箱に移動するようです．必要に応じて完全に削除して下さい）
DST_DIR_NAME = 'AI_advanced_temp'

# tempフォルダをまるごとgoogle driveに退避
MOUNT_POINT = '/content/drive'
DST_DIR_PATH = os.path.join(MOUNT_POINT, 'MyDrive', DST_DIR_NAME)
if not os.path.isdir(MOUNT_POINT):
    drive.mount(MOUNT_POINT)
if os.path.exists(DST_DIR_PATH):
    !rm -r $DST_DIR_PATH
!cp -r temp $DST_DIR_PATH

##### 学習済みニューラルネットワークモデルのロード

In [ ]:
import torch


# ニューラルネットワークモデルとその学習済みパラメータをファイルからロード
gen_model = Generator(C=C, H=H, W=W, N=N, K=len(TARGET_ATTRIBUTES))
gen_model.load_state_dict(torch.load(MODEL_FILE_G, weights_only=True))
#gen_model.load_state_dict(torch.load(autosaved_model_name(MODEL_FILE_G, 90), weights_only=True)) # 例えば90エポック目のモデルをロードしたい場合は，このようにする

##### テスト処理1
- 正規分布に従って複数の乱数ベクトルをランダムサンプリングし，それをデコーダに通して画像を生成．属性ラベルは固定値で指定

In [ ]:
import torch
from mylib.data_io import show_images, to_sigmoid_image


gen_model = gen_model.to(DEVICE)
gen_model.eval()

# 生成する画像の枚数
n_gen = 32

# 属性ラベルの指定値
# このサンプルコードでは TARGET_ATTRIBUTES = ['Blond_Hair', 'Brown_Hair', 'Black_Hair', 'Gray_Hair', 'Eyeglasses', 'Male', 'Young'] と設定しているので，
#   'Blond_Hair' = 0, # ブロンド髪ではない
#   'Brown_Hair' = 0, # 茶髪ではない
#   'Black_Hair' = 1, # 黒髪である
#   'Gray_Hair'  = 0, # 白髪ではない
#   'Eyeglasses' = 0, # 眼鏡やサングラスをかけていない
#   'Male'       = 0, # 男性でない（== 女性）
#   'Young'      = 1, # 若い
# という意味になる
attributes = [0, 0, 1, 0, 0, 0, 1]

# 標準正規分布 N(0, 1) に従って適当に乱数ベクトルを作成
Z = torch.randn((n_gen, N)).to(DEVICE)

# 属性ラベル情報の作成
L = torch.tensor([attributes], dtype=torch.float32).repeat((n_gen, 1)).to(DEVICE)

# 乱数ベクトルと属性ラベルをデコーダに入力し，その結果を表示
with torch.inference_mode():
    Y = gen_model(Z, L)
    Y = to_sigmoid_image(Y)
    show_images(Y.to('cpu').detach(), num=n_gen, num_per_row=8, title='CGAN_sample_generated_case1', save_fig=True)

##### テスト処理2
- 乱数ベクトルを一つだけサンプリングし，それをデコーダに通して画像を生成．属性ラベルは，一つの次元を徐々に変化させる形で指定

In [ ]:
import copy
import torch
from mylib.data_io import show_images, to_sigmoid_image


gen_model = gen_model.to(DEVICE)
gen_model.eval()

# ベースとなる属性ラベル
base_attributes = [0, 0, 1, 0, 0, 0, 1]

# 上の属性ラベルのうち何番目の属性値を変化させるか
# 以下の例は
#   - 0番目の属性（ 'Blond_Hair', ベース値 0 ）を 0 から 1 に徐々に変化
#   - 2番目の属性（ 'Black_Hair', ベース値 1 ）を 1 から 0 に徐々に変化
# という意味になり，すなわち，ブロンド髪から黒髪への属性変化に相当
targets = [0, 2]

# 生成する画像の枚数
n_gen = 16 # 上で指定した属性ラベルを 0～1 の間で n_gen 段階に変化させる

# 標準正規分布 N(0, 1) に従って適当に乱数ベクトルを作成
Z = torch.randn((1, N)).repeat((n_gen, 1)).to(DEVICE)

# 属性ラベル情報の作成
L = []
for i in range(n_gen):
    attributes = copy.deepcopy(base_attributes)
    for t in targets:
        # t番目の属性値を 0〜1 の範囲でずらす
        if base_attributes[t] == 0:
            attributes[t] = i / (n_gen - 1)
        else:
            attributes[t] = 1 - i / (n_gen - 1)
    L.append(attributes)
L = torch.tensor(L, dtype=torch.float32).to(DEVICE)

# 乱数ベクトルと属性ラベルをデコーダに入力し，その結果を表示
with torch.inference_mode():
    Y = gen_model(Z, L)
    Y = to_sigmoid_image(Y)
    show_images(Y.to('cpu').detach(), num=n_gen, num_per_row=8, title='CGAN_sample_generated_case2', save_fig=True)